In Python, you can create classes dynmically.  Since classes are objects, they can be constructed at runtime.  There are several approaches to accomplish this.  The simplest way is by creating a `dataclass`, a more general by using the `type` function creatively.

## Dynamic data classes

In [1]:
from dataclasses import make_dataclass
import typing

The `dataclasses` module defines the `make_dataclass` function that allows you to create data classes on the fly in a very straightforward way.

As a first example, you can create the class `Animal` that we will use as a base class for other classes.

In [2]:
Animal = make_dataclass('Animal', [('name', str), ('nr_legs', int), ('species', str)])

As this is a concrete class, it can be used to instantiate objects of the type `types.Animal`.

In [3]:
nyx = Animal(name='Nyx', species='cat', nr_legs=4)

In [4]:
type(nyx)

types.Animal

In [5]:
del nyx

### Derived classes

You can also use `Animal` as a base class for other classes, in this case `Cat` and `Dog`.

In [6]:
Cat = make_dataclass('Cat', [('species', str, 'cat'), ('nr_legs', int, 4), ('sound', str, 'meow')], bases=(Animal, ))

In [7]:
nyx = Cat(name='Nyx')

In [8]:
nyx.sound

'meow'

In [9]:
nyx.species

'cat'

In [10]:
nyx.nr_legs

4

In [11]:
Dog = make_dataclass('Dog', [('nr_legs', int, 4), ('species', str, 'dog'), ('sound', str, 'woof')], bases=(Animal, ))

In [12]:
pluto = Dog(name='Pluto')

In [13]:
pluto.species

'dog'

In [14]:
pluto.name

'Pluto'

In [15]:
type(pluto)

types.Dog

In [16]:
isinstance(pluto, Animal)

True

In [17]:
del nyx, pluto

### Adding methods

In [18]:
def make_sound(self):
    print('woof')

In [19]:
setattr(Dog, 'make_sound', make_sound)

In [20]:
lassie = Dog(name='Lassie')

In [21]:
lassie.make_sound()

woof


In [22]:
del lassie
del Animal, Cat

## Using `type`

Using the `type` function to create new classes is slightly more involved, but offers  more flexibility.  Specifically, it is possible to add methods to classes easily.

The function bedow will serve as the `__init__` function for a class you can construcut using `type`.  Whereas `__init__` is automatically generated when you use data classes, for classes generated using `type` this has to be done manually.

In [23]:
def animal_init(self, name, species=None, nr_legs=None):
    self.name = name
    self.species = species
    self.nr_legs = nr_legs

The class `Animal` below is similar to that created using `make_dataclass`, but without the autogeneraed dunder methods.

In [24]:
Animal = type('Animal', tuple(), {'name': None, 'species': None, 'nr_legs': None, '__init__': animal_init})

In [25]:
nyx = Animal(name='Nyx')

In [26]:
nyx.name

'Nyx'

In [27]:
nyx

Note that the `Animal` class generating using `type` doesn't have a `__repr__` method as opposed to a data class.

In [28]:
def animal_repr(self):
    return '(' + ', '.join(f'{key}={val}' for key, val in self.__dict__.items()) + ')'

In [29]:
setattr(Animal, '__repr__', animal_repr)

In [30]:
seth = Animal(name='Seth', species='cat', nr_legs=4)

In [31]:
seth

(name=Seth, species=cat, nr_legs=4)

This method will show all the object attributes, so those defined in the objects `__dict__` attribute, not those defined as class attributes.

In [32]:
del nyx, seth

### Derived classes

As with data classes, you can create a subclass of `Animal` by mentioning it as a base class in the call to `type`.  You can also define additional class attributes, e.g., `sound`.

In [33]:
Cat = type('Cat', (Animal, ), {'name': None, 'species': 'cat', 'nr_legs': 4, 'sound': 'meow'})

In [34]:
nyx = Cat(name='Seth')

In [35]:
nyx.sound

'meow'

In [36]:
nyx

(name=Seth, species=None, nr_legs=None)

Note that although you specified the number of legs as a class attribute, this will be overridden in `Animal`s `__init__` method.  Hence you can define an `__init__` method that calls the one defined in the parent class to initializes the object's attributes it inherits from the parent class.  The attributes specific to the `Cat` class are initialized from the class attribute, `sound` in this case.

In [37]:
def cat_init(self, name):
    attrs = {key: val for key, val in Cat.__dict__.items() if not key.startswith('__') and
             val is not None and key in Animal.__dict__}
    Animal.__init__(self, name=name, **attrs)
    self.sound = self.__class__.sound

In [38]:
setattr(Cat, '__init__', cat_init)

In [39]:
seth = Cat(name='Nyx')

In [40]:
seth

(name=Nyx, species=cat, nr_legs=4, sound=meow)

In [41]:
del nyx, seth

### Adding methods

In [42]:
nyx = Cat(name='Nyx')

In [43]:
nyx.sound

'meow'

As before, we can add a method to the class.

In [44]:
def cat_sound(self):
    print(f'cat says {self.sound}')

In [45]:
setattr(Cat, 'make_sound', cat_sound)

In [46]:
nyx.make_sound()

cat says meow


It is also possbile to add a method to a specific object, rather than a class.

In [47]:
seth = Cat(name='seth')

In [54]:
def seth_sound(self=seth):
    print(f'{self.name} says mraw')

In [55]:
setattr(seth, 'make_sound', seth_sound)

In [56]:
seth.make_sound()

seth says mraw


In [57]:
nyx.make_sound()

cat says meow


Note that the `seth_sound` function has a default argument set to the object it will be added to.  This can be generalized by defining a higher order function.

In [62]:
def add_sound(obj):
    def sound_func():
        print(f'{obj.name} says meow as well')
    setattr(obj, 'make_sound', sound_func)

In [63]:
isis = Cat(name='Isis')

In [64]:
isis.make_sound()

cat says meow


In [65]:
add_sound(isis)

In [66]:
isis.make_sound()

Isis says meow as well


Obviously, methods can dynamically be added to any Python class or object, this is often referred to as monkey pathcing.